# AWS Bedrock Access & Quota Checker

This notebook helps troubleshoot AWS Bedrock access, particularly for Claude models.

**IMPORTANT**: Before committing to GitHub:
1. Clear all cell outputs that contain AWS account info
2. Run `assume` or similar AWS credential manager first
3. Check no sensitive data in outputs (account IDs, ARNs, keys)

## Setup Requirements
```bash
pip install boto3 anthropic tabulate python-dotenv
```

In [ ]:
import boto3
import json
from datetime import datetime
from typing import Dict, List, Any
from tabulate import tabulate

# Optional: Load .env if not using assume/granted
# from dotenv import load_dotenv
# load_dotenv()

In [ ]:
def verify_aws_identity():
    """Verify AWS credentials and identity
    NOTE: Output will contain sensitive data - clear before commit
    """
    sts = boto3.client('sts')
    try:
        identity = sts.get_caller_identity()
        print("AWS Identity Verified:")
        print(f"User Type: {identity['Arn'].split(':')[5].split('/')[0]}")
        return True
    except Exception as e:
        print(f"AWS Authentication Error: {e}")
        return False

# Verify AWS access
verify_aws_identity()

In [ ]:
def get_bedrock_regions() -> List[str]:
    """Returns list of AWS regions where Bedrock is available"""
    return [
        "us-east-1",    # N. Virginia
        "us-east-2",    # Ohio
        "us-west-2",    # Oregon
        "ap-northeast-1",# Tokyo
        "ap-southeast-1",# Singapore
        "ap-southeast-2",# Sydney
        "eu-central-1", # Frankfurt
        "eu-west-1"     # Ireland
    ]

def check_bedrock_model(region: str = 'us-east-1', model_id: str = 'anthropic.claude-3-sonnet-20240229-v1:0') -> Dict:
    """Check specific model availability and status
    NOTE: Output contains account-specific info - clear before commit
    """
    bedrock = boto3.client('bedrock', region_name=region)
    try:
        response = bedrock.get_foundation_model(modelIdentifier=model_id)
        return {
            'status': response['modelDetails']['modelLifecycle']['status'],
            'types': response['modelDetails']['inferenceTypesSupported'],
            'streaming': response['modelDetails']['responseStreamingSupported']
        }
    except Exception as e:
        return {'error': str(e)}

# Test basic model access
print("Checking Claude 3 Sonnet access...")
check_bedrock_model()

In [ ]:
def check_region_quotas(region: str) -> Dict[str, Any]:
    """Check Bedrock quotas for a region
    NOTE: Quota values are account-specific - clear output before commit
    """
    quotas = boto3.client('service-quotas', region_name=region)
    try:
        response = quotas.list_service_quotas(ServiceCode='bedrock')
        relevant_quotas = []
        for quota in response['Quotas']:
            if any(term in quota['QuotaName'].lower() for term in 
                  ['invoke', 'token', 'anthropic', 'claude']):
                relevant_quotas.append({
                    'name': quota['QuotaName'],
                    'value': quota['Value'],
                    'adjustable': quota['Adjustable']
                })
        return relevant_quotas
    except Exception as e:
        return [{'error': str(e)}]

# Check quotas in main regions
main_regions = ['us-east-1', 'us-west-2', 'ap-northeast-1']
for region in main_regions:
    print(f"\nQuotas in {region}:")
    quotas = check_region_quotas(region)
    for q in quotas:
        print(f"- {q.get('name', 'ERROR')}: {q.get('value', 'N/A')}")

In [ ]:
def test_model_invocation(region: str = 'us-east-1', 
                         model_id: str = 'anthropic.claude-instant-v1',
                         max_retries: int = 3,
                         delay: int = 30):
    """Test actual model invocation with retry logic"""
    from anthropic import AnthropicBedrock
    import time
    
    client = AnthropicBedrock(aws_region=region)
    
    for attempt in range(max_retries):
        try:
            response = client.messages.create(
                model=model_id,
                max_tokens=256,
                messages=[
                    {
                        "role": "user",
                        "content": "Say hello!"
                    }
                ]
            )
            return {"success": True, "response": response.content[0].text}
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            if attempt < max_retries - 1:
                print(f"Waiting {delay} seconds...")
                time.sleep(delay)
    
    return {"success": False, "error": "Max retries exceeded"}

# Test with different models
models_to_test = [
    "anthropic.claude-instant-v1",
    "anthropic.claude-3-sonnet-20240229-v1:0",
    "anthropic.claude-3-5-sonnet-20240620-v1:0"
]

for model in models_to_test:
    print(f"\nTesting {model}...")
    result = test_model_invocation(model_id=model)
    if result['success']:
        print("Success!")
    else:
        print(f"Failed: {result.get('error')}")

## Common Issues & Solutions

1. Throttling (429 errors):
   - Check quotas in different regions
   - Implement exponential backoff
   - Consider using inference profiles

2. Access Denied (403 errors):
   - Verify IAM permissions
   - Check model access is enabled in console
   - Verify AWS credentials

3. Invalid Model ID:
   - Double-check model identifiers
   - Verify model availability in region
   - Check for cross-region inference requirements

## Before Committing
1. Clear all cell outputs
2. Verify no sensitive data in any remaining outputs
3. Remove any account-specific configurations